# Data SAO 158379

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.time import Time

import scipy.special as sc
from scipy.interpolate import interp1d
from astropy import units as u
from skyfield.api import load
import pymc as pm
import arviz as az

import aesara
import aesara.tensor as at
from aesara.compile.ops import as_op
az.style.use("arviz-darkgrid")
# plt.style.use(['science', 'notebook', 'grid'])
pd.set_option('display.float_format',  '{:}'.format)

In [ ]:
df = pd.read_csv("sao-158379.csv", sep=',')
df.head()

In [ ]:
# hilangkan karakter [ dan ] dari data
df['timeInfo'] = df['timeInfo'].map((lambda x: x.lstrip('[').rstrip(']')))

# ubah ke format waktu dan ambil tanggal dan jamnya
df['timeInfo'] = pd.to_datetime(df['timeInfo'])

df.head()

In [ ]:
data = df[['FrameNum','timeInfo','signal-target']]
data.columns = ['Frame','Time', 'Value']
data.head()

In [ ]:
data['JD'] = Time(data['Time']).jd

data['frac_JD'] = data['JD'] - np.floor(data['JD']).astype(int)
data

In [ ]:
# import matplotlib.dates as mdates

# fig, ax = plt.subplots(figsize=(15,5))
# ax.plot(data['Time'], data['Value'], 'ro', alpha=0.5)
# #ax.plot(data['Frame'], data['Value'], 'ro', alpha=0.35)
# # ax.set_xlabel('UTC (mm:ss.ssssss)')
# #ax.set_xlim(3300, 3700)
# ax.set_xlabel('Frames')

# date_form = mdates.DateFormatter("%H:%M:%S")#, tz=pytz.timezone(‘Australia/Sydney’))
# #ax.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
# #ax.xaxis.set_major_locator(mdates.HourLocator(byhour=None, interval=2))
# ax.xaxis.set_major_formatter(date_form)
# #ax.tick_params(‘x’, labelrotation=45)

# # data['Time'].dt.hour()

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
# ax.plot(data['Time'], data['Value'], 'ro', alpha=0.5)
ax.plot(data['Frame'], data['Value'], 'ro', alpha=0.35)
# ax.set_xlabel('UTC (mm:ss.ssssss)')
ax.set_xlim(3300, 3700)
ax.set_xlabel('Frames')
# fig, ax = plt.subplots(figsize=(15,5))
# ax.plot(data['frac_JD'], data['Value'], 'ro', alpha=0.5)
# ax.set_xlabel('Frac JD')

### Normalisasi Data

In [ ]:
value_scaled = data['Value']/(data['Value'].std()*2)

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
# ax.plot(data['frac_JD'], value_scaled, 'ro', alpha=0.35)
ax.plot(data['Frame'], value_scaled, 'ro', alpha=0.35)
ax.set_xlabel('Frames')

### Kurva Benda Hitam untuk SAO 158379

#### Cari temperatur efektif

Temperatur efektif bintang bisa didekati dengan persamaan yang menghubungkan indeks warna bintang, $B - V$ dengan temperatur efektif bintang, $T_\mathrm{eff}$. Persamaan yang digunakan dalam pekerjaan ini diadopsi dari pekerjaan **Ballesteros 2012** (https://arxiv.org/pdf/1201.1809.pdf) sebagai berikut:

$$
T_\mathrm{eff} =  4600\left(\frac{1}{0.92 (B-V) + 1.7} + \frac{1}{0.92 (B-V) + 0.62} \right)
$$

In [ ]:
# Satuan t_eff adalah Kelvin

def ballesteros(bv):
    return 4600. * ((1./(0.92*bv + 1.7)) + (1./(0.92*bv + 0.62)))

In [ ]:
b = 6.95
v = 6.85

bv = b - v

teff = ballesteros(bv)
teff

In [ ]:
def Planck(λ, T): 
    """
    λ: dalam cm
    T: dalam K
    """
    a = 3.7412e-5 / ((λ * 1e-7) **5) # 1e-7 adalah konversi dari nm ke cm
    b = np.exp(1.43879/(λ * 1e-7 * T)) - 1
    return a/b

In [ ]:
x_axis = np.arange(100, 2000)

In [ ]:
y_axis = Planck(x_axis, teff)

In [ ]:
plt.plot(x_axis, y_axis)

In [ ]:
den = np.max(y_axis)

y_val = y_axis/den

In [ ]:
plt.plot(x_axis, y_val)

### Kurva transmisi QHY 174M GPS *unfiltered*

In [ ]:
df = pd.read_csv('qhy174_response.csv', delimiter=',', header='infer')
df.head()

In [ ]:
x = df['wavelength']
y = df['response']
plt.plot(x, y, 'r')

In [ ]:
bandwidth = np.arange(330, 1100, 10) # rentang sesuai kurva transmisi

bandwidth

In [ ]:
f_interp = interp1d(x, y, kind='cubic')

### Jarak Bulan saat pengamatan

Pengamatan dilakukan pada tanggal **8 Juli 2022 pukul 12:57:10.8 UT**. Ephemeris diambil dari https://ssd.jpl.nasa.gov/ftp/eph/planets/ascii/

In [ ]:
# load timescale dan tampilkan waktu saat ini
ts = load.timescale()

# custom waktu untuk yyyy, mm, dd, hh, mm, ss
t = ts.utc(2022, 7, 8, 12, 57, 10.8)

In [ ]:
# Load the JPL ephemeris DE430 (covers 1549-2650) dan cari jarak Bumi-Bulan
planets = load('de430.bsp')
earth, moon = planets['earth'], planets['moon']

In [ ]:
# posisi Bulan dari Bumi
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()

In [ ]:
# tampilkan koordinat Bulan dan jarak Bulan dalam km
print(ra)
print(dec)
print(distance.km)

In [ ]:
d_moon = distance.km
d_moon

### Inisiasi model

In [ ]:
x_data = np.array(data['Frame'])
y_data = np.array(data['Value'])
# y_data = np.array(value_scaled)

In [ ]:
def fresnel_polikromatik(x, x0, d, A):
    y0 = np.zeros(len(x))
    for i in range (len(bandwidth)):
        w = (x - x0) * np.sqrt(2 / (bandwidth[i] * d)) * np.sqrt(1. / 1e-6) # faktor konversi ke satuan meter
        # model untuk disappearance
        cc, ss = sc.fresnel(-w)
        y = A * 0.5 * ((cc + 0.5)**2 + (ss + 0.5)**2)
        y_new = f_interp(bandwidth[i]) * (Planck(bandwidth[i], T)/den) * y
        # y_new = f_interp(bandwidth[i]) * y
        y0 += y_new
    return y0

In [ ]:
# Variabel benar
# -------------
x0_true = 3480
# x0_true = 0.039705
A_true = 1.
σ_true = 150.
T = teff

In [ ]:
y_syn = fresnel_polikromatik(x_data, x0_true, d_moon, A_true)
y_syn = np.random.normal(y_syn, scale=σ_true)

y_syn_scaled = y_syn / (y_syn.std()*2)

In [ ]:
plt.figure(figsize = (12,5), facecolor='w')
plt.scatter(x_data, y_syn_scaled, marker='o', c='r', alpha=0.35)
# plt.invert_yaxis()
# plt.xlim(3300, 3700)
plt.xlabel('Frames')
plt.ylabel('Intensity')

In [ ]:
@as_op(itypes=[at.dvector, at.dscalar, at.dscalar, at.dscalar], otypes=[at.dvector])
def t_fresnel_polikromatik(x, x0, d, A):
    y0 = np.zeros(len(x))
    for i in range (len(bandwidth)):
        w = (x - x0) * np.sqrt(2 / (bandwidth[i] * d)) * np.sqrt(1. / 1e-6) # faktor konversi ke satuan meter
        cc, ss = sc.fresnel(-w)
        y = A * 0.5 * ((cc + 0.5)**2 + (ss + 0.5)**2)
        y_new = f_interp(bandwidth[i]) * (Planck(bandwidth[i], T)/den) * y
        y0 += y_new
    return y0

In [ ]:
x_shared = aesara.shared(x_data)
d_shared = aesara.shared(d_moon)

with pm.Model() as model_polikromatik:
    A = pm.Normal('A', mu=1., sigma=1.)
    σ = pm.HalfCauchy('σ', beta=1.)
    x0 = pm.Normal('x0', mu=3480., sigma=1.)
    
    μ = t_fresnel_polikromatik(x_shared, x0, d_shared, A)
    y_pred = pm.Normal('y_pred', mu=μ, sigma=σ, observed=value_scaled)
    
pm.model_to_graphviz(model_polikromatik)

In [ ]:
with model_polikromatik:
    idata = pm.sample(cores=1)
    pm.sample_posterior_predictive(idata, extend_inferencedata=True)

In [ ]:
az.plot_trace(idata)

In [ ]:
az.summary(idata)

In [ ]:
idata

In [ ]:
idata_pp = idata.posterior_predictive

idata_pp_st = idata_pp.stack(sample=("chain", "draw"))

crit_l = np.percentile(idata_pp_st['y_pred'], 2.5, axis=1)  # 2.5% persentil 
crit_u = np.percentile(idata_pp_st['y_pred'], 97.5, axis=1)  # 97.5% persentil
mean_spp = np.mean(idata_pp_st['y_pred'], axis=1) # Median

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(x_data, mean_spp, linestyle='-', linewidth=1, color="#5500ff")
plt.fill_between(x_data, crit_l, crit_u, alpha=0.2, color="#00cc66")
plt.scatter(x_data, value_scaled, c='r', marker='o', alpha=0.3)
# plt.plot(x_data, y_data, 'r-', lw=1, alpha=0.5)
plt.xlabel('Frames')
plt.xlim(3300, 3700)
plt.ylabel('Intensity')
plt.tight_layout()
plt.legend(['Mean Posterior Prediction', '95% Posterior Prediction', 'Simulated Data'], loc='upper left')
# plt.savefig('hasil-kurvatransmisi-planck.png', dpi=150)

### *Data smoothing*

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
y_filtered = savgol_filter(value_scaled, 10, 2)

In [ ]:
plt.figure(figsize=(15,5))
plt.scatter(x_data, y_filtered, c='r', marker='o', alpha=0.3)
plt.xlim(3300, 3700)

In [ ]:
with pm.Model() as model_polikromatik_smt:
    A = pm.Normal('A', mu=1., sigma=1.)
    σ = pm.HalfCauchy('σ', beta=1.)
    x0 = pm.Normal('x0', mu=3480., sigma=1.)
    
    μ = t_fresnel_polikromatik(x_shared, x0, d_shared, A)
    y_pred = pm.Normal('y_pred', mu=μ, sigma=σ, observed=y_filtered)
    
pm.model_to_graphviz(model_polikromatik_smt)

In [ ]:
with model_polikromatik_smt:
    idata = pm.sample(cores=1)
    pm.sample_posterior_predictive(idata, extend_inferencedata=True)

In [ ]:
az.plot_trace(idata)

In [ ]:
az.summary(idata)

In [ ]:
idata_pp = idata.posterior_predictive

idata_pp_st = idata_pp.stack(sample=("chain", "draw"))

crit_l = np.percentile(idata_pp_st['y_pred'], 2.5, axis=1)  # 2.5% persentil 
crit_u = np.percentile(idata_pp_st['y_pred'], 97.5, axis=1)  # 97.5% persentil
mean_spp = np.mean(idata_pp_st['y_pred'], axis=1) # Median

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(x_data, mean_spp, linestyle='-', linewidth=1, color="#5500ff")
plt.fill_between(x_data, crit_l, crit_u, alpha=0.2, color="#00cc66")
plt.scatter(x_data, y_filtered, c='r', marker='o', alpha=0.3)
# plt.plot(x_data, y_data, 'r-', lw=1, alpha=0.5)
plt.xlabel('Frames')
plt.xlim(3300, 3700)
plt.ylabel('Intensity')
plt.tight_layout()
plt.legend(['Mean Posterior Prediction', '95% Posterior Prediction', 'Simulated Data'], loc='upper left')
# plt.savefig('hasil-kurvatransmisi-planck.png', dpi=150)

### Konversi *Frame Number* ke waktu

In [ ]:
x = data['Frame']
y = data['JD']

plt.plot(x, y)
plt.xlabel('Frame #')
plt.ylabel('Julian Date')

In [ ]:
f = interp1d(x, y, kind='cubic')

#### Ambil nilai dari `arviz inference data`

In [ ]:
framenum = idata.posterior.mean().x0.values

In [ ]:
# interpolasi

date = f(framenum)
type(date)

### Ubah `ndarray` ke `str`

In [ ]:
date_str = np.array_str(date)
date_str

In [ ]:
# Ubah ke format waktu

j_date = Time(date_str, format='jd')
# j_date.value
iso_time = j_date.iso
iso_time

In [ ]:
data.dtypes

### Ubah `str` ke format `timestamp`

In [ ]:
t_iso_time = pd.to_datetime(iso_time)
t_iso_time.time()

In [ ]:
import matplotlib.dates as mdates

fig, ax = plt.subplots(figsize=(15,5))
ax.plot(data['Time'], mean_spp, linestyle='-', linewidth=1, color="#5500ff")
ax.fill_between(data['Time'], crit_l, crit_u, alpha=0.2, color="#00cc66")
ax.plot(data['Time'], value_scaled, 'ro', alpha=0.35)

plt.axvline(x = t_iso_time, color = 'k', lw=3, ls='--', label = t_iso_time.time())
plt.legend(loc = 'upper right')

#ax.plot(data['Frame'], data['Value'], 'ro', alpha=0.35)
# ax.set_xlabel('UTC (mm:ss.ssssss)')
ax.set_xlim(data['Time'][580], data['Time'][790]) # baris ke-# untuk kolom 'Time'
ax.set_xlabel('Time (UTC)')

date_form = mdates.DateFormatter("%H:%M:%S")#, tz=pytz.timezone(‘Australia/Sydney’))
#ax.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
#ax.xaxis.set_major_locator(mdates.HourLocator(byhour=None, interval=2))
ax.xaxis.set_major_formatter(date_form)
# ax.tick_params('x', labelrotation=45)

# data['Time'].dt.hour()

In [ ]:
az.plot_ppc(idata, num_pp_samples=100, kind='cumulative')